In [ ]:
import torch
import torch.nn.functional as F

def ppo_loss(pi_new, pi_old, v_now, r, v_next, eps=0.2, c1=0.5, c2=0.01, gamma=0.99):
    """
    PPO核心损失计算（变量极简+逻辑清晰）
    输入：
        pi_new: 新策略输出的动作log概率 (batch_size,)
        pi_old: 旧策略输出的动作log概率 (batch_size,)
        v_now: 当前状态价值估计 (batch_size,)
        r: 即时奖励 (batch_size,)
        v_next: 下一个状态价值估计 (batch_size,)
        eps: 裁剪系数（默认0.2）
        c1: 价值损失权重（默认0.5）
        c2: 熵正则权重（默认0.01）
        gamma: 折扣因子（默认0.99）
    输出：
        total_loss: PPO总损失
    """
    # 1. 计算优势值A（TD误差近似，也可替换为GAE）
    td_target = r + gamma * v_next  # TD目标（真实长期回报近似）
    A = td_target - v_now           # 优势值（动作好坏的衡量）
    
    # 2. 策略损失（带裁剪的重要性采样）
    ratio = torch.exp(pi_new - pi_old)  # 重要性比：pi_new/pi_old（log转exp）
    clip_ratio = torch.clamp(ratio, 1-eps, 1+eps)  # 裁剪到[0.8,1.2]
    surr1 = ratio * A                  # 未裁剪项
    surr2 = clip_ratio * A             # 裁剪项
    policy_loss = -torch.mean(torch.min(surr1, surr2))  # 取min+负号（梯度下降）
    
    # 3. 价值损失（MSE拟合TD目标）
    value_loss = c1 * F.mse_loss(v_now, td_target)
    
    # 4. 熵正则（鼓励探索，可选但推荐）
    entropy = -torch.mean(pi_new)  # 策略熵（log概率的负期望，熵越大探索性越强）
    entropy_loss = -c2 * entropy   # 负号：最小化损失=最大化熵
    
    # 5. 总损失
    total_loss = policy_loss + value_loss + entropy_loss
    
    return total_loss

In [ ]:
'''
Author: yewang0628 wangye374127@gmail.com
Date: 2025-11-13 13:50:38
LastEditors: yewang0628 wangye374127@gmail.com
LastEditTime: 2025-11-13 14:17:13
FilePath: \vs_code\练手.ipynb
Description: 这是默认设置,请设置`customMade`, 打开koroFileHeader查看配置 进行设置: https://github.com/OBKoro1/koro1FileHeader/wiki/%E9%85%8D%E7%BD%AE
'''
# PPO GAE版
import torch
import torch.nn.functional as F

def gae_advantage(v_now, v_next, r, gamma=0.99, gae_lambda=0.95):
    """
    计算GAE（广义优势估计），单独抽离函数，逻辑更清晰
    输入：
        v_now: 当前状态价值 (batch_size,) 或 (traj_len,)
        v_next: 下一个状态价值 (batch_size,) 或 (traj_len,)
        r: 即时奖励 (batch_size,) 或 (traj_len,)
        gamma: 折扣因子（与主函数一致）
        gae_lambda: GAE参数（默认0.95，平衡偏差-方差）
    输出：
        A: GAE优势值 (batch_size,) 或 (traj_len,)
    """
    # 1. 计算时序差分残差（TD Error）
    td_residual = r + gamma * v_next - v_now  # δ_t = r_t + γV(s_{t+1}) - V(s_t)
    
    # 2. 反向累加计算GAE（从最后一个时间步往前推）
    A = torch.zeros_like(td_residual)
    advantage = 0.0  # 初始优势值（最后一个时间步之后无后续，优势为0）
    # 反向遍历：从后往前累加 (γλ)^k * δ_{t+k}
    for t in reversed(range(len(td_residual))):
        advantage = td_residual[t] + gamma * gae_lambda * advantage
        A[t] = advantage
    
    # 可选：优势值标准化（减少训练波动，推荐添加）
    A = (A - A.mean()) / (A.std() + 1e-8)  # 加1e-8避免除零
    return A

def ppo_loss(pi_new, pi_old, v_now, r, v_next, eps=0.2, c1=0.5, c2=0.01, gamma=0.99, gae_lambda=0.95):
    """
    PPO核心损失计算（GAE优势值版，变量极简+逻辑清晰）
    输入新增：
        gae_lambda: GAE参数（默认0.95，工业界常用值）
    其他输入/输出与原版本一致
    """
    # 1. 计算GAE优势值（替换原TD误差近似）
    A = gae_advantage(v_now, v_next, r, gamma, gae_lambda)
    
    # 2. 策略损失（带裁剪的重要性采样，逻辑不变）
    ratio = torch.exp(pi_new - pi_old)  # 重要性比：pi_new/pi_old（log转exp，数值稳定）
    clip_ratio = torch.clamp(ratio, 1-eps, 1+eps)  # 裁剪到[0.8,1.2]
    surr1 = ratio * A
    surr2 = clip_ratio * A
    policy_loss = -torch.mean(torch.min(surr1, surr2))  # 取min+负号（梯度下降适配）
    
    # 3. 价值损失（MSE拟合TD目标，逻辑不变）
    td_target = r + gamma * v_next  # TD目标依然用于价值网络拟合
    value_loss = c1 * F.mse_loss(v_now, td_target)
    
    # 4. 熵正则（鼓励探索，逻辑不变）
    entropy = -torch.mean(pi_new)  # 离散动作策略熵（log概率的负期望）
    entropy_loss = -c2 * entropy
    
    # 5. 总损失
    total_loss = policy_loss + value_loss + entropy_loss
    
    return total_loss



In [1]:

#DPOloss
import torch
import torch.nn.functional as F

def dpo_loss(pi_w, pi_l, beta=0.1):
    """
    DPO 核心损失（变量极简版）
    输入：
        pi_w: 新策略对「赢样本w」的log概率 (batch_size,) —— 人类偏好的回复
        pi_l: 新策略对「输样本l」的log概率 (batch_size,) —— 人类不偏好的回复
        beta: 温度超参（控制偏好强度，默认0.1，工业界常用）
    输出：
        dpo_total_loss: DPO总损失
    """
    # 1. 计算偏好加权后的log概率（beta放大偏好差异）
    w_score = beta * pi_w
    l_score = beta * pi_l
    
    # 2. 对比损失：让赢样本的相对概率最大化（交叉熵形式）
    # 等价于 -log( exp(w_score) / (exp(w_score) + exp(l_score)) )
    logits = torch.stack([w_score, l_score], dim=1)  # (batch, 2)
    loss = -torch.mean(F.log_softmax(logits, dim=1)[:, 0])  # 只关注赢样本的softmax概率
    
    # 3. 可选：熵正则（鼓励探索，可选添加）
    entropy = -torch.mean(pi_w)  # 用赢样本的熵近似（简化处理）
    entropy_loss = 0.01 * entropy  # 权重0.01，弱正则
    
    return loss - entropy_loss  # 负号：最小化损失=最大化熵

if __name__=='__mian__':
    # 模拟batch_size=32的偏好数据（赢/输样本的log概率）
    batch_size = 32
    pi_w = torch.randn(batch_size)  # 赢样本log概率
    pi_l = torch.randn(batch_size)  # 输样本log概率

    loss = dpo_loss(pi_w, pi_l)
    print("DPO损失:", loss.item())

In [ ]:
#GRPO loss

import torch
import torch.nn.functional as F

def grpo_loss(pi_new, pi_old, r, group_size=4, eps=0.2, c2=0.01):
    """
    GRPO 核心损失（变量极简版，无Critic，组内对比估算优势）
    输入：
        pi_new: 新策略的log概率 (batch_size × group_size,) —— 组内所有样本的logp
        pi_old: 旧策略的log概率 (batch_size × group_size,) —— 固定不更新
        r: 组内样本的奖励/偏好分数 (batch_size × group_size,) —— 组内每条回复的得分
        group_size: 每组采样数量（默认4，常用值）
        eps: 裁剪系数（同PPO，默认0.2）
        c2: 熵正则权重（默认0.01）
    输出：
        grpo_total_loss: GRPO总损失
    """
    # 1. 重构维度：适应组内计算 (batch_size, group_size)
    batch_size = pi_new.shape[0] // group_size
    pi_new = pi_new.reshape(batch_size, group_size)
    pi_old = pi_old.reshape(batch_size, group_size)
    r = r.reshape(batch_size, group_size)
    
    # 2. 组内相对优势估算（替代Critic，核心！）
    # 优势 = 样本奖励 - 组内平均奖励（组内对比，体现相对好坏）
    r_mean = r.mean(dim=1, keepdim=True)  # 每组的平均奖励
    A = r - r_mean  # 组内相对优势（无需Critic，直接通过组内对比获得）
    
    # 3. 策略损失（带裁剪的重要性采样，同PPO核心逻辑）
    ratio = torch.exp(pi_new - pi_old)  # 重要性比（log转exp，数值稳定）
    clip_ratio = torch.clamp(ratio, 1-eps, 1+eps)  # 限制更新幅度
    surr1 = ratio * A
    surr2 = clip_ratio * A
    policy_loss = -torch.mean(torch.min(surr1, surr2))  # 取min+负号（梯度下降）
    
    # 4. 熵正则（鼓励组内多样性，可选但推荐）
    entropy = -torch.mean(pi_new, dim=1)  # 每组的熵
    entropy_loss = -c2 * entropy.mean()  # 批量平均
    
    # 5. 总损失（无价值损失，因为去除了Critic）
    total_loss = policy_loss + entropy_loss
    
    return total_loss

if __name__=='__main__':
# 模拟输入：batch_size=8组，每组4个样本（total=32个样本）
    batch_size = 8
    group_size = 4
    total_samples = batch_size * group_size

    pi_new = torch.randn(total_samples)  # 新策略log概率
    pi_old = torch.randn(total_samples)  # 旧策略log概率（固定）
    r = torch.randn(total_samples)       # 组内样本奖励（可正可负）

    loss = grpo_loss(pi_new, pi_old, r, group_size=group_size)
    print("GRPO总损失:", loss.item())

In [ ]:
import torch
import torch.nn.functional as F

def grpo_loss(pi_new, pi_old, r, group_size=4, eps=0.2, c2=0.01, std_eps=1e-8):
    """
    GRPO 核心损失（带标准差归一化的优势计算）
    输入：
        pi_new: 新策略的log概率 (batch_size × group_size,) —— 组内所有样本的logp
        pi_old: 旧策略的log概率 (batch_size × group_size,) —— 固定不更新
        r: 组内样本的奖励/偏好分数 (batch_size × group_size,) —— 组内每条回复的得分
        group_size: 每组采样数量（默认4，常用值）
        eps: 裁剪系数（同PPO，默认0.2）
        c2: 熵正则权重（默认0.01）
        std_eps: 防止除以零的小常数（默认1e-8）
    输出：
        grpo_total_loss: GRPO总损失
    """
    # 1. 重构维度：适应组内计算 (batch_size, group_size)
    batch_size = pi_new.shape[0] // group_size
    pi_new = pi_new.reshape(batch_size, group_size)
    pi_old = pi_old.reshape(batch_size, group_size)
    r = r.reshape(batch_size, group_size)
    
    # 2. 组内相对优势估算（带标准差归一化）
    r_mean = r.mean(dim=1, keepdim=True)  # 每组的平均奖励
    r_std = r.std(dim=1, keepdim=True)    # 每组的奖励标准差
    # 优势 = (样本奖励 - 组内平均奖励) / 组内标准差（标准化处理）
    A = (r - r_mean) / (r_std + std_eps)  # 加入std_eps防止除以零
    
    # 3. 策略损失（带裁剪的重要性采样，同PPO核心逻辑）
    ratio = torch.exp(pi_new - pi_old)  # 重要性比（log转exp，数值稳定）
    clip_ratio = torch.clamp(ratio, 1-eps, 1+eps)  # 限制更新幅度
    surr1 = ratio * A
    surr2 = clip_ratio * A
    policy_loss = -torch.mean(torch.min(surr1, surr2))  # 取min+负号（梯度下降）
    
    # 4. 熵正则（鼓励组内多样性，可选但推荐）
    entropy = -torch.mean(pi_new, dim=1)  # 每组的熵
    entropy_loss = -c2 * entropy.mean()  # 批量平均
    
    # 5. 总损失（无价值损失，因为去除了Critic）
    total_loss = policy_loss + entropy_loss
    
    return total_loss

if __name__=='__main__':
    # 模拟输入：batch_size=8组，每组4个样本（total=32个样本）
    batch_size = 8
    group_size = 4
    total_samples = batch_size * group_size

    pi_new = torch.randn(total_samples)  # 新策略log概率
    pi_old = torch.randn(total_samples)  # 旧策略log概率（固定）
    r = torch.randn(total_samples)       # 组内样本奖励（可正可负）

    loss = grpo_loss(pi_new, pi_old, r, group_size=group_size)
    print("带标准差归一化的GRPO总损失:", loss.item())
